## Scipy and Optimization

In the last notebook we took a tour of the `numpy` and `scipy` stack as it relates to math and statistics. This notebook is going to focus on another subset of `scipy` functionality that is absolutely critical to data analytics. Nearly every statistical algorithm aside from Ordinary Least Squares (OLS) regression cannot be solved algebraically. Because they cannot be solved algebraically, they must be solved **numerically**.

### Numeric Optimization

Let's start with an example of optimization. Let's say that you know the demand function for tickets to watch a local sports franchise. You can write the inverse demand function as

$$ P = 300 - \frac{1}{2}Q $$

and the total cost function as

$$ TC = 4000 + 45Q $$

In order to choose the right number of tickets to sell, you need to calculate the quantity of tickets that will maximize profits. We can calculate total revenue as $ TR = P \times Q $, and we can calculate profit as $ \Pi = TR - TC $. This means that our profit function is

$$ \Pi = 300Q - \frac{1}{2}Q^2 - 4000 - 45Q $$

In order to find the $Q$ associated with the highest achievable level of profit, we can use calculus to find the point at which the rate of change in the profit function is zero ($\frac{\partial\Pi}{\partial Q}=0$).

$$ \frac{\partial\Pi}{\partial Q} = 300 - Q - 45 = 0 \implies Q = 255$$

So we can **algebraically** solve this particular problem. This isn't always the case. Using `scipy`, we can solve this same problem, as well as many algebraically intractable problems that might be more interesting to us.


<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/paraboloid.png" width="200" height="200" />


In any optimization problem, we need to find a way to get ourselves to the minimum, and to know when we get there. When we look at the above image, we are able to visually trace the functional shape (looks like a rainbow ice cream cone to me...) and locate the bottom of the function. What we want to do is utilize an algorithm to "trace" our way from an arbitrary starting point within a function to the optimal point in that function.

In three or fewer dimensions, this is easy. Regressions and statistical models often live in worlds with 100's or 1000's (even millions sometimes) of dimensions. We can't visualize our way to the bottom of those functions!

The class of algorithm that is used to solve these problems is called **gradient descent**.

<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/gradDesc.png" width="400" />

**Gradient descent** is an algorithm that explores the shape of the function, and determines which direction is most likely to lead to the optimal point. Let's focus on minimization. We want to find our way to the *bottom* of a function, and we can use gradient descent to try to get there. Given any starting point, our goal is to check the direction in which we can move downward most quickly, and start moving in that direction. At some distance from our starting point, we will stop and re-evaluate the direction in which we would like to travel. Are we still heading downhill in the steepest direction? If we aren't, then we need to update our behavior.

Our gradient descent algorithm will keep "looking around" and moving down until it reaches a point at which it can no longer move "down" in any meaningful way. That is the stopping point, and is treated as the optimum.


With an intuitive understanding of how optimization will happen computationally, it's time to learn a bit more about the math and the code that will help us to achieve computational optimization.

Consider a function, $f$, with two variables $x$ and $y$. Because there are two input variables in the function, it has two partial derivatives:

$$ \frac{\partial f}{\partial x} \text{ and } \frac{\partial f}{\partial y} $$

Each partial derivative tells us how $f$ changes as we move in a particular dimension **all else equal**. The **gradient**, then, is the vector of all partial derivatives of a given function at any point along the function:

$$ \nabla f = \left[ \begin{matrix} \frac{\partial f}{\partial x} \\ \\ \frac{\partial f}{\partial y} \end{matrix} \right]  $$

We can use the gradient to determine the linear approximation of a function at any given point. Think about the gradient as the mathematical representation of the slope and direction of a hill you are hiking on. If we know the gradient, we know which way is down. As we continue to calculate gradients while walking, we can continue to ensure that we will walk downhill until we reach the bottom of the hill.




The steps of our gradient descent function will be the following:

- Evaluate the gradient of the function
- Find the direction of steepest descent
- Determine how far to move in that direction
- Move to new point
- Reevaluate the gradient
- Stop moving when gradient is within a margin of error from 0

Let's try to implement gradient descent by solving our old profit maximization problem computationally. The very first thing that we need to do is write a Python function that will represent our mathematical function.

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    return tr - tc

This function will allow us to calculate profit at any output level based on our assumed total costs and demand curve. With this function, we can quickly calculate the gradient (in this case, just a simple derivative because our function is univariate) by calculating profit at two nearby points, and dividing by the horizontal distance between those points:

In [ ]:
# Gradient at q=200

(profit(201) - profit(199))/2

    55.0



Thus, a one unit increase in output at $Q=200$ results in a $55 increase in profits. This is cool, but it isn't enough for us to find the point of maximum profit (the optimal point). For that, we will need to calculate LOTS of gradients in order to move along the function until we cannot increase profits any further.

Fortunately for us, `scipy` comes with optimization tools that will do all of the heavy lifting of the "search" for the optimal point. All that we need to do is frame our question algorithmically, and let `scipy` do the rest:

In [ ]:
import numpy as np
from scipy.optimize import minimize

We start by importing the `minimize` function from `scipy.optimize`. Hang on! Weren't we working on a MAXIMIZATION problem?? What are we doing here?

Maximization and minimization are the **same thing**. To maximize a function, you can multiply that function by `-1` and then calculate the minimum of the new "upside-down" function. It is functionally equivalent. So, in computational optimization, we always minimize.

### Prepping for optimization

As we prepare to optimize, there are two common problems with our function that we may need to resolve:

1) When using `minimize` we can only pass an array of inputs, so we have to be careful to write our function accordingly
2) Our problem is concave, and so has a maximum
	- We need to restate it as a minimization problem

Problem 1 does not apply, since our function in univariate. In order to make our problem a minimization problem, we can flip our profit maximization function. We will simply return -1 * profit:

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    pi =  tr - tc # didn't name it profit because that is our function's name! Don't want to clutter our name space!
    return -1*pi

### Making the call to `minimize`

Now that our function is ready, it is time to minimize! The `minimize` function takes two arguments:
1. Our function that we want to optimize
2. A starting guess (as a vector)

Let's give it a try.

In [ ]:
res = minimize(profit, [0]) # provide function and starting inputs
res

          fun: -28512.499999980355
     hess_inv: array([[1.00000175]])
          jac: array([0.])
      message: 'Optimization terminated successfully.'
         nfev: 21
          nit: 3
         njev: 7
       status: 0
      success: True
            x: array([255.00019821])



That's it! No calculus, no searching, no checking gradients manually. `minimize` simply takes our function and our starting guess and brings us back the optimal choice. We get lots of information stored in the attributes of the `res` object:

- `fun` provides the value of the function (this is -1 times the profit level at the optimal output in our example)
- `hess_inv` and `jac` are measures of gradient and are used to determine how far to go and in which direction
- `message` should be self-explanatory
- `nfev` is the number of times the function (in this case `profit`) was evaluated during the search
- `nit` is the number of iterations it took to find the optimum
- `njev` is the number of times the Jacobian was estimated
- `status` is a code associated with the `message` and `success` atrributes
- `success` tells you whether or not an optimum was found (sometimes it cannot be easily found!)
- `x` probably the most important attribute. This tells us the optimal input value (in our case $Q$), or set of values depending on our function. In a regression context, this could represent the fitted coefficients!

Going forward, you will realize (especially because so many of them print the output as they optimize) just how many libraries in Python use this minimize function behind the scenes. It is used in `statsmodels`, `sklearn`, and many other high-profile libraries! Now that you know where it really happens (in `scipy`!), you'll be better able to troubleshoot the problems that will inevitably arise as you use statistical models.

**Solve-it!**

In this lesson we learned about optimization using SciPy. For the assignment this week, I would like you to build off of your `RegressionModel` class. You will add a Logistic Regression (Logit) method to your class, so that when the `regression_type` parameter is `logit`, Logistic Regression Results are returned.

Your job is to create the following functionality within your class object:
- a method (call it `logistic_regression`) that estimates the results of logistic regression using your `x` and `y` data frames, and using a likelihood function and gradient descent (DO NOT USE PREBUILT REGRESSION FUNCTIONS).
    - You need to write a function to calculate the Log-likelihood of your model
    - You need to implement gradient descent to find the optimal values of beta
    - You need to use your beta estimates, the model variance, and calculate the standard errors of the coefficients, as well as Z statistics and p-values
    - the results should be stored in a dictionary named `results`, where each variable name (including the intercept if `create_intercept` is `True`) is the key, and the value is another dictionary, with keys for `coefficient`, `standard_error`, `z_stat`, and `p_value`. The coefficient should be the log odds-ratio (which takes the place of the coefficients in OLS)
- a method called `fit_model` that uses the `self.regression_type` attribute to determine whether or not to run an OLS or Logistic Regression using the data provided. This method should call the correct regression method.
- an updated method (call it `summary`) that presents your regression results in a table
    - Columns should be: Variable name, Log odds-ratio value, standard error, z-statistic, and p-value, in that order.
    - Your summary table should have different column titles for OLS and Logistic Regressions! (think if statement...)

You only need to define the class. My code will create an instance of your class (be sure all the names match these instructions, and those from last week!), and provide data to run a regression. I will provide the same data to you, so that you can experiment and make sure that your code is functioning properly.

NOTE: I have created a [primer on Logistic regression](https://github.com/dustywhite7/Econ8320/blob/master/SlidesPDF/9-2%20-%20Logit%20Primer.pdf) to go with this assignment. See the Github slidesPDF folder

Put the code that you would like graded in the cell labeled `#si-exercise`. I recommend copying your code from the last assignment (in chapter 9 about linear regression and `numpy`), and continuing from there.



In [ ]:
#si-exercise

#si-exercise
import numpy as np
from scipy.stats import t
import scipy.stats as stats
import pandas as pd
from scipy.optimize import minimize

class RegressionModel:
  """
  course 10, 10-2
  adding Logistic regression to the previous OLS regression
  version:4-good
  """

  def __init__(self, x, y, create_intercept=True, regression_type="ols"):
    #print (f'****** type of y \n\n{y}\n\n')
    self.x = pd.DataFrame(x)
    self.y = y #type(y): <class 'pandas.core.series.Series'> not <class 'pandas.core.frame.DataFrame'>
    self.y2 = pd.DataFrame(y)
    self.num_obs = len(self.x) #number of observation
    _, self.num_vars = x.shape
    if isinstance(regression_type, str) and regression_type in ["ols","logit"]:
      self.regression_type = regression_type
    else:
      raise RuntimeError("Enter ols or logit to processed")

    #>exogenous and endogenous variables labels
    self.x_list = list(self.x.columns.values)
    self.y_label = y.name
    #print (f'y_label = {self.y_label}, x_list = {self.x_list}\n***********************')

    #>convert DataFrame to Matrixes but keep x&y type <class 'pandas.core.frame.DataFrame'> koz unitest
    self.add_intercept(create_intercept)
    self.x1 = np.array(self.x)
    self.y1 = np.array(self.y2)
    #print(self.x1)

#> Helpful Functions, Begining
  def add_intercept(self, create_intercept):
    """will add a column of ones to the data frame x if create_intercept is True"""
    if create_intercept:
      ##> Update Endogenous Variables first elt with <intercept>
      self.x_list.append('intercept')
      #print (type(self.x_list))
      ##> Set beta_0, all 1 in First Column of x
      all_1 = pd.DataFrame([1 for i in range(self.num_obs)], columns=['intercept'])
      self.x = pd.concat([self.x, all_1], axis=1).reset_index(drop=True)
      #update shape
      _, self.num_vars = self.x.shape

  def inver(self, mat):
    """inverse of the matrix"""
    return np.linalg.inv(mat)
#> Helpful Functions, End

  def BetaHat(self):
    """>Solving Beta_hat
       >y = x . Beta_hat + E
       >Beta_hat = ( x’ x)_inv  x’ y
    """
    xt_x = self.x1.T @ self.x1
    xt_y = self.x1.T @ self.y1
    self.Beta_hat_M = self.inver(xt_x) @ (xt_y)
    self.Beta_hat = list(self.Beta_hat_M.T)[0] #from (5,1) to a list

#> Col3 - Begining: standard error
  def chat2(self): #chat or shat, sound about the same.
    """Variance <s**2>  = [ (y - x * B_hat)_tansp * (y - x * B_hat) ] / ( n - k )  """
    n, k = self.x1.shape
    self.df = n - k
    res = ((self.y1 - self.x1 @ self.Beta_hat_M).T @ (self.y1 - self.x1 @ self.Beta_hat_M)) / ( self.df)
    #print(f'chat2: {res}')
    return float(res[0][0])

  def varianceOfBeta_hat(self):
    #""" Covariance = chat2 * ( xt * x )_inver """
    covariance = self.chat2() * ( self.inver( self.x1.T @ self.x1 ) )
    return np.diag(covariance)

  def std_error(self):
    self.stdError = list(np.sqrt(self.varianceOfBeta_hat()))
    # https://youtu.be/e8MyE5iRBEA?si=ZaJSQTf36KptTSwT&t=398
#> Col3 - End: standard error

#> Col4 - Begining: t-statistic
  def t_statistic(self):
    """ t =  Beta_hat(j) / stdError(j), each element one-by-one """
    #self.t_stat =  self.Beta_hat_M / self.stdError
    self.t_stat = [Bj / Oj for Bj, Oj in zip(self.Beta_hat, self.stdError)]
#> Col4 - End: t-statistic

#> Col5 - Begining: p-value
  def p_value(self):
    """ p_val =  t.sf ( t_stat , degreeOfFreedom_df) """
    self.p_val = t.sf (self.t_stat, self.df)
#> Col5 - End

#> Set the dict for Results
  def resultToDict(self):
    dfResultsDict = pd.DataFrame({
            "Variable name"     : self.x_list
            ,"coefficient": self.Beta_hat
            ,"standard_error"   : self.stdError
            ,"t_stat"      : self.t_stat
            ,"p_value"          : self.p_val
                    })
    dfResultsDict = dfResultsDict.set_index("Variable name")
    self.results = dfResultsDict.transpose().to_dict()

#> End of dict Method

  def ols_regression(self):
    """ Main method to call"""
    self.BetaHat()
    self.varianceOfBeta_hat()
    self.std_error()
    self.t_statistic()
    self.p_value()
    self.resultToDict()

  #############################################################
  ##
  ## Exo - 10,
  ##
  ##############################################################

  def fit_model(self):
    if self.regression_type == 'ols':
      self.ols_regression()
    elif self.regression_type == 'logit':
      self.logistic_regression()

##-- end ex 10


  def summary_ols(self):
    #for OLS
    """Will display the summary of all feature of the Multi dimension Regression"""
    dfResults = pd.DataFrame({
                    "Variable name"     : self.x_list
                    ,"coefficient value": self.Beta_hat
                    ,"standard error"   : self.stdError
                    ,"t-statistic"      : self.t_stat
                    ,"p-value"          : self.p_val
                           })
    print(f'"\n-------- Summary --------\n"{dfResults}')

  def logistic_regression(self):
    def loglike(b, y, x):
      #print(f'A) shapes x: {x.shape}, shapes y: {y.shape} and b:{b}')
      #print(f'A1) y.iloc[5] [{y.loc[5]}]')
      gamma = np.exp( x @ b) / (1 + np.exp( x @ b))
      #print (f'B) gamma.shape: {gamma.shape}')
      ll = -1 * sum(yi * np.log(gi) + (1 - yi) * np.log(1 - gi) \
                    for yi,gi in zip(np.array(y).ravel(), gamma))
      return ll

    #b=[0.086] * (self.num_vars)
    b = np.full((self.num_vars), 0.1)
    y=self.y2
    x=self.x

    betas = minimize(loglike, b, args=(y,x)).x  ; self.Beta_hat = betas

    ## Others dependant results
    shat2 = self.num_obs * np.mean(y) * (1-np.mean(y))
    cov = shat2 * np.linalg.inv(self.x.T @ self.x)
    variances = cov.diagonal()
    self.stdError = np.sqrt(np.diag(cov))
    self.zScores = betas / self.stdError
    self.p_val = stats.norm.sf(np.abs(self.zScores)) * 2

    self.results = {}
    variable_names = self.x.columns
    for i, name in enumerate(variable_names):
      self.results[name] = {
        'coefficient': betas[i],
        'standard_error': self.stdError[i],
        'z_stat': self.zScores[i],
        'p_value': self.p_val[i]
      }

    #print(self.results)

  def summary(self):
    if  self.regression_type == 'logit':
      dfResults = pd.DataFrame({
                      "Variable name"     : self.x_list
                      ,"coefficient value": self.Beta_hat
                      ,"standard error"   : self.stdError
                      ,"z-statistic"      : self.zScores
                      ,"p-value"          : self.p_val
                            })
    elif self.regression_type == 'ols':
      dfResults = pd.DataFrame({
                    "Variable name"     : self.x_list
                    ,"coefficient value": self.Beta_hat
                    ,"standard error"   : self.stdError
                    ,"t-statistic"      : self.t_stat
                    ,"p-value"          : self.p_val
                           })
    print(f'"\n-------- Summary --------\n"{dfResults}')

In [ ]:

## testing my code
data = pd.read_csv("assignment8Data_2.csv")
x = data.loc[:100, ['sex','age','educ']]
y = data.loc[:100, 'white']

#reg = RegressionModel(x, y, True, 'ols')
reg = RegressionModel(x, y, True, 'logit')
reg.fit_model()
reg.summary()

"
-------- Summary --------
"  Variable name  coefficient value  standard error  z-statistic       p-value
0           sex          -1.122915        0.397988    -2.821482  4.780229e-03
1           age          -0.007013        0.010836    -0.647167  5.175242e-01
2          educ          -0.046486        0.101003    -0.460246  6.453399e-01
3     intercept           5.735443        1.126621     5.090837  3.564855e-07


      sex = {'coefficient': -1.1229156890097627,
      'standard_error': 0.39798772782618025,
      'z_stat': -2.821483202869492,
      'p_value': 0.004780214077269219}
      age = {'coefficient': -0.007012518056833769,
      'standard_error': 0.010835821823286998,
      'z_stat': -0.6471607019011091,
      'p_value': 0.5175279421902776}
      educ = {'coefficient': -0.046485475816343394,
      'standard_error': 0.10100278092776117,
      'z_stat': -0.46023956359766527,
      'p_value': 0.6453442758780246}
      intercept = {'coefficient': 5.735435005488546,
      'standard_error': 1.1266207023561843,
      'z_stat': 5.090830475148922,
      'p_value': 3.56498650369634e-07}

In [ ]:
a = [0]*4
b = np.zeros(4,int)
b.shape
np.array(a)

In [ ]:
## testing my code

import numpy as np
from scipy.stats import t
import pandas as pd

data = pd.read_csv("assignment8Data_2.csv")

x = data.loc[:100, ['sex','age','educ']]
y = data.loc[:100, 'white']

reg = RegressionModel(x, y, True, 'logit')
reg.fit_model()


betas
